In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import h5py
%matplotlib inline

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters):
    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [3]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters,s=2):

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###
    return X


In [4]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape=(64, 64, 3), classes=6):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024],s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X)

    return model

In [5]:
model = ResNet50(input_shape=(64, 64, 3), classes=1)

In [6]:
NAME="RESNET501"
tensorboard=TensorBoard(log_dir="logs/{}".format(NAME))
filepath="RESNET501.hdf5"    
modelcheckpoint=ModelCheckpoint(filepath,monitor="val_acc",save_best_only=True,mode="max",verbose=1)
callbacks_list=[tensorboard,modelcheckpoint]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
#Now loading the data using the h5py file
testFileName="test_catvnoncat.h5"
fileTest=h5py.File(testFileName,"r")
trainFileName="train_catvnoncat.h5"
fileTrain=h5py.File(trainFileName,"r")
X_test=fileTest["test_set_x"].value
Y_test=fileTest["test_set_y"].value
X_train=fileTrain["train_set_x"].value
Y_train=fileTrain["train_set_y"].value
#prininting the shapes and standardizing the datasets
print("The total number of training examples are {}".format(X_train.shape[0]))
print("The total number of testing examples are {}".format(X_test.shape[0]))
print("The shape of Features for training set is {}".format(X_train.shape))
print("The shape of Labels for training set is {}".format(Y_train.shape))
print("The shape of Features for testing set is {}".format(X_test.shape))
print("The shape of Labels for testing set is {}".format(Y_test.shape))
#Now having mean=0
X_train=X_train/255
X_test=X_test/255

The total number of training examples are 209
The total number of testing examples are 50
The shape of Features for training set is (209, 64, 64, 3)
The shape of Labels for training set is (209,)
The shape of Features for testing set is (50, 64, 64, 3)
The shape of Labels for testing set is (50,)


In [8]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 32,shuffle=True,validation_split=0.1,callbacks=callbacks_list)

Train on 188 samples, validate on 21 samples
Epoch 1/100
160/188 [========================>.....] - ETA: 4s - loss: 1.7490 - acc: 0.6125 
Epoch 00001: val_acc improved from -inf to 0.80952, saving model to RESNET501.hdf5
188/188 [==============================] - 53s 284ms/step - loss: 1.6736 - acc: 0.6277 - val_loss: 0.5935 - val_acc: 0.8095
Epoch 2/100
160/188 [========================>.....] - ETA: 0s - loss: 0.8169 - acc: 0.6375
Epoch 00002: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 32ms/step - loss: 0.8004 - acc: 0.6383 - val_loss: 0.5463 - val_acc: 0.8095
Epoch 3/100
160/188 [========================>.....] - ETA: 0s - loss: 0.6907 - acc: 0.7188
Epoch 00003: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 0.6699 - acc: 0.7128 - val_loss: 0.5244 - val_acc: 0.8095
Epoch 4/100
160/188 [========================>.....] - ETA: 0s - loss: 0.2822 - acc: 0.8938
Epoch 00004: val_acc did not improv

Epoch 32/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0093 - acc: 1.0000
Epoch 00032: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 0.0120 - acc: 1.0000 - val_loss: 2.1345 - val_acc: 0.1429
Epoch 33/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0949 - acc: 0.9750
Epoch 00033: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 0.0833 - acc: 0.9787 - val_loss: 1.1488 - val_acc: 0.5238
Epoch 34/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0308 - acc: 0.9875
Epoch 00034: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 31ms/step - loss: 0.0276 - acc: 0.9894 - val_loss: 1.9087 - val_acc: 0.2857
Epoch 35/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0068 - acc: 1.0000
Epoch 00035: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss

Epoch 63/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0086 - acc: 0.9938
Epoch 00063: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 0.0076 - acc: 0.9947 - val_loss: 2.1261 - val_acc: 0.5238
Epoch 64/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0056 - acc: 1.0000
Epoch 00064: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 0.0061 - acc: 1.0000 - val_loss: 2.2946 - val_acc: 0.5714
Epoch 65/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0036 - acc: 1.0000
Epoch 00065: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 2.6973 - val_acc: 0.5714
Epoch 66/100
160/188 [========================>.....] - ETA: 0s - loss: 0.0015 - acc: 1.0000
Epoch 00066: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 31ms/step - loss

160/188 [========================>.....] - ETA: 0s - loss: 4.6754e-04 - acc: 1.0000
Epoch 00093: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 4.0183e-04 - acc: 1.0000 - val_loss: 1.8733 - val_acc: 0.6190
Epoch 94/100
160/188 [========================>.....] - ETA: 0s - loss: 2.5208e-05 - acc: 1.0000
Epoch 00094: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 2.4103e-05 - acc: 1.0000 - val_loss: 1.8367 - val_acc: 0.6667
Epoch 95/100
160/188 [========================>.....] - ETA: 0s - loss: 5.8966e-05 - acc: 1.0000
Epoch 00095: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 30ms/step - loss: 5.5275e-05 - acc: 1.0000 - val_loss: 1.8178 - val_acc: 0.7143
Epoch 96/100
160/188 [========================>.....] - ETA: 0s - loss: 3.8974e-05 - acc: 1.0000
Epoch 00096: val_acc did not improve from 0.80952
188/188 [==============================] - 6s 3

In [9]:
preds = model.evaluate(X_test, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

50/50 [==============================] - 1s 24ms/step
Loss = 1.6306120872497558
Test Accuracy = 0.7000000095367431


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation